# Chatting with GPT4all

GPT4ALL can be used for chat conversations by using local LLM. 

Use the GPT4All chat_session context manager to hold chat conversations with the model.

For more details see the [documentation](https://docs.gpt4all.io/gpt4all_python.html#chatting-with-gpt4all). 

In [1]:
# load envirenmental variables from .env file 
import os 
import openai
import textwrap
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file 


In [2]:
from gpt4all import GPT4All

## Load model

In [3]:
llama_model_name = "llama-2-7b-chat.ggmlv3.q4_0.bin"

# initialization of the models
model  = GPT4All(llama_model_name,  model_path="models")


Found model file at  models\\llama-2-7b-chat.ggmlv3.q4_0.bin


## Chatting with GPT4All

In [8]:
with model.chat_session():
    response1 = model.generate(prompt='hello', temp=0)
    response2 = model.generate(prompt='write me a short poem', temp=0)
    response3 = model.generate(prompt='Why did the chicken cross the road?', temp=0)
    print(model.current_chat_session)

[{'role': 'system', 'content': '### System:\nYou are an AI assistant that follows instruction extremely well. Help as much as you can.'}, {'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': ' Hello! How may I assist you today?'}, {'role': 'user', 'content': 'write me a short poem'}, {'role': 'assistant', 'content': " The sky is blue, so bright and true,\nA gentle breeze blows, so cool and new.\nThe sun shines down, with all its might,\nAnd birds take to the wing, without a sound.\nNature's beauty, in this moment,\nIs truly something to behold, and feel."}, {'role': 'user', 'content': 'Why did the chicken cross the road?'}, {'role': 'assistant', 'content': ' To get to the other side, of course!'}]


## Managing Templates

In [4]:

system_template = 'You are friendly chatbot.'
# many models use triple hash '###' for keywords, Vicunas are simpler:
prompt_template = 'USER: {0}\nASSISTANT: '
with model.chat_session(system_template, prompt_template):
    response1 = model.generate('Hi, my name is Lisa')
    print(response1)
    print()
    response2 = model.generate('Can you remind me, what is name given in my eralier prompt?')
    print(response2)


 Hello Lisa! It's nice to meet you. How can I assist you today? Is there something on your mind that you would like to talk about or ask me?

 Of course! Your name was mentioned earlier as "Lisa".


In [16]:
system_template = 'A chat between a curious user and an artificial intelligence assistant.'
prompt_template = 'USER: {0}\nASSISTANT: '
prompts = ['name 3 colors', 'now name 3 fruits', 'what were the 3 colors in your earlier response?']
first_input = system_template + prompt_template.format(prompts[0])
response = model.generate(first_input, temp=0)
print(response)
for prompt in prompts[1:]:
    response = model.generate(prompt_template.format(prompt), temp=0)
    print(response)

 Great! I'm happy to help you with that. Can you please provide me with more information about the colors you are interested in? For example, are they bright or muted, warm or cool, etc.?
 Sure! Here are three types of fruit: apples, bananas, and oranges.
 In my previous response, I provided a list of 10 colors that are commonly associated with the brand Coca-Cola. These colors are: red and white.


In [5]:
from itertools import cycle

class RotatingTemplateGPT4All(GPT4All):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._templates = [
            "Respond like a pirate.",
            "Respond like a politician.",
            "Respond like a philosopher.",
            "Respond like a Klingon.",
        ]
        self._cycling_templates = cycle(self._templates)

    def _format_chat_prompt_template(
        self,
        messages: list,
        default_prompt_header: str = "",
        default_prompt_footer: str = "",
    ) -> str:
        full_prompt = default_prompt_header + "\n\n" if default_prompt_header != "" else ""
        for message in messages:
            if message["role"] == "user":
                user_message = f"USER: {message['content']} {next(self._cycling_templates)}\n"
                full_prompt += user_message
            if message["role"] == "assistant":
                assistant_message = f"ASSISTANT: {message['content']}\n"
                full_prompt += assistant_message
        full_prompt += "\n\n" + default_prompt_footer if default_prompt_footer != "" else ""
        print(full_prompt)
        return full_prompt

In [8]:
model = RotatingTemplateGPT4All(llama_model_name,  model_path="models")
with model.chat_session():  # starting a session is optional in this example
    response1 = model.generate("hi, who are you?")
    print(response1)
    print()
    response2 = model.generate("what can you tell me about snakes?")
    print(response2)
    print()
    response3 = model.generate("what's your opinion on Chess?")
    print(response3)
    print()
    response4 = model.generate("tell me about ancient Rome.")
    print(response4)

Found model file at  models\\llama-2-7b-chat.ggmlv3.q4_0.bin
[INST]<<SYS>>You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.<</SYS>>[/INST]

USER: hi, who are you? Respond like a pirate.


HAPPY ASSISTANT: Arrrr, shiver me timbers! *adjusts eye patch* I be Happy the Assistant, here to help ye with any questions or tasks ye may have! Yer welcome to ask me anything, and I'll do me bestest to assist ye in a safe and respectful manner. So hoist the sails and set sail on yer query, matey!

USER: what can you tell me about snakes? Respond like 

# Introspection

In [10]:
import logging

logging.basicConfig(level=logging.INFO)
model = GPT4All('nous-hermes-13b.ggmlv3.q4_0.bin', model_path="models")
with model.chat_session('You are a geography expert.\nBe terse.',
                        '### Instruction:\n{0}\n### Response:\n'):
    response = model.generate('who are you?', temp=0)
    print(response)
    response = model.generate('what are your favorite 3 mountains?', temp=0)
    print(response)

100%|██████████| 7.32G/7.32G [1:57:48<00:00, 1.04MiB/s] 


Model downloaded at:  models\\nous-hermes-13b.ggmlv3.q4_0.bin


INFO:gpt4all.pyllmodel:LLModel.prompt_model -- prompt:
You are a geography expert.
Be terse.

### Instruction:
who are you?
### Response:

===/LLModel.prompt_model -- prompt/===
INFO:gpt4all.pyllmodel:LLModel.prompt_model -- prompt:
### Instruction:
what are your favorite 3 mountains?
### Response:

===/LLModel.prompt_model -- prompt/===


I am an AI-powered chatbot designed to assist users with their queries related to geographical information.
1) Mount Everest - Located in the Himalayas, it is the highest mountain on Earth and a significant challenge for mountaineers.
2) Kangchenjunga - This mountain is located in the Himalayas and is the third-highest peak in the world after Mount Everest and K2.
3) Lhotse - Located in the Himalayas, it is the fourth highest mountain on Earth and offers a challenging climb for experienced mountaineers.


# Interrupting Generation

In [11]:
from gpt4all import GPT4All
model = GPT4All('orca-mini-3b.ggmlv3.q4_0.bin', model_path="models")

def stop_on_token_callback(token_id, token_string):
    # one sentence is enough:
    if '.' in token_string:
        return False
    else:
        return True

response = model.generate('Blue Whales are the biggest animal to ever inhabit the Earth.',
                          temp=0, callback=stop_on_token_callback)
print(response)

Found model file at  models\\orca-mini-3b.ggmlv3.q4_0.bin


INFO:gpt4all.pyllmodel:LLModel.prompt_model -- prompt:
Blue Whales are the biggest animal to ever inhabit the Earth.
===/LLModel.prompt_model -- prompt/===


 They can grow up to 100 feet (30 meters) long and weigh as much as 20 tons (18 metric tons).
